In [85]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype

from collections import Counter

In [87]:
path = './data/'

data = pd.read_csv(path + 'train.csv')

print(len(data))
print(data.columns)

1647
Index(['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type',
       'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine',
       'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity'],
      dtype='object')


In [88]:
data.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,BMW,3-Series 320d,800000,2012,75576,Diesel,Automatic,Mumbai,White,Second,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BMW,X1 sDrive20d xLine,2199000,2016,77000,Diesel,Automatic,Surat,Black,First,Individual,1995 cc,184 bhp @ 4000 rpm,350 Nm @ 1750 rpm,RWD,4454.0,2044.0,1545.0,5.0,63.0
2,Mahindra,XUV500 W4 1.99,800000,2017,112000,Diesel,Manual,Muzaffurpur,Silver,First,Individual,1997 cc,138 bhp @ 3750 rpm,320 Nm @ 1600 rpm,FWD,4585.0,1890.0,1785.0,7.0,70.0
3,Mercedes-Benz,GLS 400d 4MATIC,12900000,2021,3000,Diesel,Automatic,Delhi,White,First,Individual,2925 cc,326 bhp @ 3600 rpm,700 Nm @ 1200 rpm,AWD,5207.0,2157.0,1823.0,7.0,90.0
4,Toyota,Fortuner 2.8 4x2 AT [2016-2020],3499000,2019,73000,Diesel,Automatic,Mumbai,White,First,Individual,2755 cc,174 bhp @ 3400 rpm,450 Nm @ 1600 rpm,RWD,4795.0,1855.0,1835.0,7.0,80.0


# PREPROCESS DATA

## 1. Cleaning

In [89]:
data.dropna(how='all')

data["Engine"] = data["Engine"].str.replace(' cc', '', regex=False).astype(float)

data[['Max Power BHP', 'Max Power RPM']] = data['Max Power'].str.extract(r'(\d+)\s*bhp\s*@\s*(\d+)\s*rpm')
data['Max Power BHP'] = pd.to_numeric(data['Max Power BHP'], errors='coerce')
data['Max Power RPM'] = pd.to_numeric(data['Max Power RPM'], errors='coerce')

data[['Max Torque Nm', 'Max Torque RPM']] = data['Max Torque'].str.extract(r'(\d+)\s*Nm\s*@\s*(\d+)\s*rpm')
data['Max Torque Nm'] = pd.to_numeric(data['Max Torque Nm'], errors='coerce')
data['Max Torque RPM'] = pd.to_numeric(data['Max Torque RPM'], errors='coerce')

data = data.drop('Max Power', axis=1)
data = data.drop('Max Torque', axis=1)



Số lượng dữ liệu bị thiếu ở các cột

In [90]:
data.isna().sum()


Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 65
Drivetrain            111
Length                 52
Width                  52
Height                 52
Seating Capacity       52
Fuel Tank Capacity     90
Max Power BHP         172
Max Power RPM         172
Max Torque Nm         168
Max Torque RPM        168
dtype: int64

## 2. Xử lí dữ liệu bị thiếu

In [91]:
def drop_na(df : pd.DataFrame):
    return df.dropna(how='any')

### 2.1 Fill missing values with mean

In [92]:
def fill_mean(df : pd.DataFrame):
    return df.apply(lambda x : x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)

In [93]:
data_mean = fill_mean(data)

In [94]:
data_mean.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

--> Các cột với kiểu dữ liệu str không thể được fill

In [95]:
data_mean = drop_na(data_mean)
data_mean.to_csv(path + 'train_mean.csv', index=False)

### 2.2 Fill missing values with median

In [96]:
def fill_median(df : pd.DataFrame):
    return df.apply(lambda x: x.fillna(x.median()) if x.dtype.kind in 'biufc' else x)

In [97]:
data_median = fill_median(data)

In [98]:
data_median.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

--> Tương tự với mean, các cột có kiểu dữ liệu str không thể được fill

In [99]:
data_median = drop_na(data_median)
data_median.to_csv(path + 'train_median.csv', index=False)

### 2.3 Fill missing values with most common value

In [100]:
def fill_mode(df : pd.DataFrame):
    return df.apply(lambda x: x.fillna(x.mode()[0]))

In [101]:
data_mode = fill_mode(data)

In [102]:
data_mode.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
Max Power BHP         0
Max Power RPM         0
Max Torque Nm         0
Max Torque RPM        0
dtype: int64

Không còn missing value

In [103]:
data_mode = drop_na(data_mode)
data_mode.to_csv(path + 'train_mode.csv', index=False)

### 2.4 K - Nearest Neighbour

In [104]:
def euclidean_distance(row1, row2):
    """
    Computes the Euclidean distance between two rows, ignoring NaN values and handling only numeric data.
    Non-numeric data is excluded from the distance calculation.
    """
    numeric_mask = pd.api.types.is_numeric_dtype(row1) and pd.api.types.is_numeric_dtype(row2)

    if numeric_mask:
        mask = ~np.isnan(row1) & ~np.isnan(row2)
        if np.any(mask):
            return np.sqrt(np.sum((row1[mask] - row2[mask]) ** 2))
        else:
            return np.inf
    else:
        return np.inf

def fill_knn(df, k=3):
    df_filled = df.copy()

    for i, row in df_filled.iterrows():
        if row.isnull().any():
            distances = []

            for j, other_row in df_filled.iterrows():
                if not other_row.isnull().any():
                    dist = euclidean_distance(row.values, other_row.values)
                    distances.append((dist, j))

            distances = sorted(distances, key=lambda x: x[0])
            nearest_neighbors = [df_filled.iloc[j] for dist, j in distances[:k]]

            for col in df_filled.columns:
                if pd.isnull(df_filled.at[i, col]):
                    neighbor_values = [neighbor[col] for neighbor in nearest_neighbors]
                    neighbor_values = [val for val in neighbor_values if not pd.isnull(val)]

                    if pd.api.types.is_numeric_dtype(df_filled[col]):
                        df_filled.at[i, col] = np.mean(neighbor_values)
                    else:
                        df_filled.at[i, col] = Counter(neighbor_values).most_common(1)[0][0]

    return df_filled

In [105]:
data_knn = fill_knn(data)

In [106]:
data_knn.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
Max Power BHP         0
Max Power RPM         0
Max Torque Nm         0
Max Torque RPM        0
dtype: int64

In [107]:
data_knn = drop_na(data_knn)
data_knn.to_csv(path + 'train_knn.csv', index=False)

### 2.5 Interpolation

In [108]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_lin = inferred_data.interpolate(method='linear', limit_direction='both')
data_interpolate_lin.isna().sum()

C:\Users\phine\AppData\Local\Temp\ipykernel_5728\80612481.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data_interpolate_lin = inferred_data.interpolate(method='linear', limit_direction='both')


Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

In [109]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_pol = inferred_data.interpolate(method='polynomial', order=2)
data_interpolate_pol.isna().sum()

C:\Users\phine\AppData\Local\Temp\ipykernel_5728\3354355801.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data_interpolate_pol = inferred_data.interpolate(method='polynomial', order=2)


Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  1
Drivetrain            111
Length                  1
Width                   1
Height                  1
Seating Capacity        1
Fuel Tank Capacity      1
Max Power BHP           1
Max Power RPM           1
Max Torque Nm           1
Max Torque RPM          1
dtype: int64

In [110]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_pad = inferred_data.interpolate(method='pad', limit=2)
data_interpolate_pad.isna().sum()

C:\Users\phine\AppData\Local\Temp\ipykernel_5728\1035036417.py:3: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_interpolate_pad = inferred_data.interpolate(method='pad', limit=2)


Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                1
Drivetrain            1
Length                1
Width                 1
Height                1
Seating Capacity      1
Fuel Tank Capacity    1
Max Power BHP         1
Max Power RPM         1
Max Torque Nm         1
Max Torque RPM        1
dtype: int64

In [111]:
data_interpolate_lin = drop_na(data_interpolate_lin)
data_interpolate_lin.to_csv(path + 'train_interpolate_lin.csv', index=False)

data_interpolate_pol = drop_na(data_interpolate_pol)
data_interpolate_pol.to_csv(path + 'train_interpolate_pol.csv', index=False)

data_interpolate_pad = drop_na(data_interpolate_pad)
data_interpolate_pad.to_csv(path + 'train_interpolate_pad.csv', index=False)

## 3. Normalizing

### 1. Min-max scaling
When to Use: Good for models requiring bounded inputs (e.g., neural networks), but less ideal if Price has extreme outliers.

### 2. Log Transformation
When to Use: Ideal for skewed data. Train the model on ```Price_log```, then exponentiate predictions ```(np.exp(predictions) - 1)``` to return to the original scale.

### 3. Standardization (Z-score Normalization)
When to Use: Suitable for most machine learning models (e.g., linear regression, SVM, neural networks) and robust to moderate outliers.

### 4. Robust Scaling
When to Use: Best when Price has significant outliers that you don’t want to remove but need to mitigate.

In [112]:
def log_norm(df: pd.DataFrame, columns=None):
    new_df = df.copy()
    if columns is None:
        columns = df.columns
    for col in columns:
        if is_numeric_dtype(new_df[col]):
            new_df[col] = np.log(new_df[col] + 1)
    return new_df

def minmax_norm(df: pd.DataFrame, columns=None):
    new_df = df.copy()
    if columns is None:
        columns = df.columns
    for col in columns:
        if is_numeric_dtype(new_df[col]):
            new_df[col] = (new_df[col] - new_df[col].min()) / (new_df[col].max() - new_df[col].min())

    return new_df

def standard_norm(df: pd.DataFrame, columns=None):
    new_df = df.copy()
    if columns is None:
        columns = df.columns
    for col in columns:
        if is_numeric_dtype(new_df[col]):
            new_df[col] = (new_df[col] - new_df[col].mean()) / new_df[col].std()
    return new_df

def robust_norm(df: pd.DataFrame, columns=None, focus=0.5):
    new_df = df.copy()
    if columns is None:
        columns = df.columns
    for col in columns:
        if is_numeric_dtype(new_df[col]):
            new_df[col] = (new_df[col] - new_df[col].median()) / (new_df[col].quantile(1 - focus/2) - new_df[col].quantile(focus/2))
    return new_df

In [113]:
log_column = ['Price', 'Kilometer']
minmax_column = ['Year', 'Seating Capacity']
standard_column = ['Engine']

In [114]:
data_mean = log_norm(data_mean, columns=log_column)
data_mean = minmax_norm(data_mean, columns=minmax_column)
data_mean = standard_norm(data_mean, columns=standard_column)

In [115]:
data_mean.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,...,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Max Power BHP,Max Power RPM,Max Torque Nm,Max Torque RPM
1,BMW,X1 sDrive20d xLine,14.603514,0.823529,11.251574,Diesel,Automatic,Surat,Black,First,...,RWD,4454.0,2044.0,1545.0,0.500000,63.0,184.0,4000.0,350.0,1750.0
2,Mahindra,XUV500 W4 1.99,13.592368,0.852941,11.626263,Diesel,Manual,Muzaffurpur,Silver,First,...,FWD,4585.0,1890.0,1785.0,0.833333,70.0,138.0,3750.0,320.0,1600.0
3,Mercedes-Benz,GLS 400d 4MATIC,16.372738,0.970588,8.006701,Diesel,Automatic,Delhi,White,First,...,AWD,5207.0,2157.0,1823.0,0.833333,90.0,326.0,3600.0,700.0,1200.0
4,Toyota,Fortuner 2.8 4x2 AT [2016-2020],15.067988,0.911765,11.198228,Diesel,Automatic,Mumbai,White,First,...,RWD,4795.0,1855.0,1835.0,0.833333,80.0,174.0,3400.0,450.0,1600.0
5,Maruti Suzuki,Vitara Brezza ZDi+ Dual Tone [2017-2018],13.369225,0.823529,10.859018,Diesel,Manual,Delhi,Blue,Second,...,FWD,3995.0,1790.0,1640.0,0.500000,48.0,89.0,4000.0,200.0,1750.0


In [116]:
data_mean.to_csv(path + 'train_mean.csv', index=False)